In [2]:
import pytezos as tz
import time
from checker_tools.client.operations import inject
from pytezos.contract.call import ContractCall
from pytezos.operation.group import OperationGroup

# Deploying Checker

We will use `alice` as admin for all of our contracts.

In [3]:
TEZOS_RPC="http://localhost:20000"

alice_secret = "edsk3QoqBuvdamxouPhin7swCvkQNgq4jP5KZPbwWNnwdZpSpJiEbq"
alice = tz.Key.from_encoded_key(alice_secret)
alice_ptz = tz.pytezos.using(TEZOS_RPC, alice)

In [4]:
tez = int(1e6)  # 1 tez = 1M mutez

In [5]:
def do_it(ptz, call, amount=None):
    """Calls an entrypoint, optionally sending an amount of mutez"""
    if amount is not None:
        call = call.with_amount(amount)
    # FIXME: not sure why it is so hard to change the source?
    # This is necessary because the contract is created with the admin's PyTezosClient
    # and we want to send the transaction from someone else
    contents = call.as_transaction().contents
    og = OperationGroup(context=ptz.context, contents=contents)
    tx = og.fill(counter=int(ptz.account()["counter"])+1).sign()
    return tx.send(min_confirmations=1)
    
    
def add_operator_fa2(ptz, address, token, token_id):
    return do_it(ptz, token.update_operators([{
        "add_operator": {
            "owner": ptz.key.public_key_hash(),
            "operator": address,
            "token_id": token_id
        }
    }]))

def add_allowance_fa12(ptz, token, spender, quantity):
    return do_it(ptz,
        token.approve({
            "spender": spender.address,
            "value": quantity
        })
    )

def now_plus_10():
    return int(time.time() + 10)

In [6]:
from pytezos.contract.interface import ContractInterface, Contract, MichelsonProgram, ExecutionContext

In [7]:
from pytezos.context.impl import ExecutionContext

In [8]:
from checker_tools.builder.config import CheckerRepo
from checker_tools.client import checker
conf = CheckerRepo("..")
conf.default_config

PosixPath('../checker.yaml')

TODO: Harbinger oracle

In [9]:
oracle_source = "../utils/mock_oracle.tz"
oracle_storage={'owner': alice.public_key_hash(), 'price': (1, 1)}   # Initial price of 1 tok/kit
oracle = checker.deploy_contract(alice_ptz, source_file=oracle_source, initial_storage=oracle_storage)

In [10]:
ctez_contracts = checker.deploy_ctez(alice_ptz, conf)
wtez = checker.deploy_wtez(alice_ptz, conf)
wctez = checker.deploy_wctez(alice_ptz, conf, ctez_fa12_address=ctez_contracts["fa12_ctez"].address)

Deploying ctez contract...
Done.
Deploying ctez FA1.2 contract...
Done.
Deploying ctez CFMM contract...
Done.
Deploying liquidity contract...
Done.
Setting liquidity address in CFMM contract...
Done.
Setting CFMM amd ctez FA1.2 addresses in ctez contract...


Loading config from ../checker.yaml


Done.
Deploying the wtez contract.


Loading config from ../checker.yaml


Done.
Tez Wrapper address: KT1FJDtuFLRjZajFWVwZZhfd2xxADmqc3qoc
Deploying the wctez contract.
Done.
wctez address: KT1JzZbLsSidp71Fq8VNHr4eq3BiDqS5U5sT


In [11]:
ch = checker.deploy_checker(alice_ptz, conf, oracle=oracle, collateral_fa2=wtez,
                            cfmm_token_fa2=wctez, ctez_cfmm=ctez_contracts["cfmm"])

Loading config from ../checker.yaml


Deploying the wrapper.
Checker address: KT1Q4HEeboEqJ7gkZ9tBrZRE5GBrw4t3CDwE
Deploying the TZIP-16 metadata.
Deploying TZIP-16 metadata: chunk 1 of 10
Deploying TZIP-16 metadata: chunk 2 of 10
Deploying TZIP-16 metadata: chunk 3 of 10
Deploying TZIP-16 metadata: chunk 4 of 10
Deploying TZIP-16 metadata: chunk 5 of 10
Deploying TZIP-16 metadata: chunk 6 of 10
Deploying TZIP-16 metadata: chunk 7 of 10
Deploying TZIP-16 metadata: chunk 8 of 10
Deploying TZIP-16 metadata: chunk 9 of 10
Deploying TZIP-16 metadata: chunk 10 of 10
Deploying: touch
  deployed: chunk 0.
  deployed: chunk 1.
  deployed: chunk 2.
  deployed: chunk 3.
Deploying: create_burrow
  deployed: chunk 0.
Deploying: deposit_collateral
  deployed: chunk 0.
Deploying: withdraw_collateral
  deployed: chunk 0.
Deploying: mint_kit
  deployed: chunk 0.
Deploying: burn_kit
  deployed: chunk 0.
Deploying: activate_burrow
  deployed: chunk 0.
Deploying: deactivate_burrow
  deployed: chunk 0.
Deploying: mark_for_liquidation
  deploy

In [12]:
ch.storage()

{'lazy_functions': 16,
 'metadata': 17,
 'deployment_state': {'sealed': {'burrows': 18,
   'cfmm': {'ctok': 1,
    'kit': 1,
    'kit_in_ctok_in_prev_block': {'den': 1, 'num': 1},
    'last_level': 47,
    'lqt': 1},
   'external_contracts': {'collateral_fa2': 'KT1FJDtuFLRjZajFWVwZZhfd2xxADmqc3qoc',
    'ctez_cfmm': 'KT1J5WbrrzxCAMx3YEDGNEEmu7YwuPvxTynj',
    'ctok_fa2': 'KT1JzZbLsSidp71Fq8VNHr4eq3BiDqS5U5sT',
    'oracle': 'KT1C1SsTheFJJD96uDFNhCqBVyrEgZvikosQ'},
   'fa2_state': {'ledger': 19, 'operators': 20},
   'last_ctez_in_tez': None,
   'last_index': None,
   'liquidation_auctions': {'avl_storage': {'last_ptr': 1, 'mem': 21},
    'burrow_slices': 22,
    'completed_auctions': None,
    'current_auction': None,
    'queued_slices': 1},
   'parameters': {'burrow_fee_index': 18446744073709551616,
    'circulating_kit': 0,
    'drift': 0,
    'drift_derivative': 0,
    'imbalance_index': 18446744073709551616,
    'index': 18446744073709551616,
    'last_touched': 1687451871,
    'ou

# A few test accounts

In order to make this simulation a little more realistic, let's create a bunch of addresses, send them some tez and have them mint various quantity of kits.

In [13]:
test_accounts = [
    {
        "address": "tz1L7zaWD1aRYBTQvSdxEdc9KDzfwG4DydDu",
        "secret": "edsk4WVyqJ112qJREwC5TERn2AmmmNMyVhwWyaLgHosC48UFV6EnMG",
        "kits_to_mint": 1000*tez,
        "wtez_to_mint": 1200*tez
    },
    {
        "address": "tz1LKyW7AzoXYiPxxS6uzKwcUtpincBZduxf",
        "secret": "edsk4JaFU4PZ7vpp1kDyeE578zwJv2XpBP7QqhSQEw3mFS3rmBP5nJ",
        "kits_to_mint": 1000*tez,
        "wtez_to_mint": 1500*tez
    },
    {
        "address": "tz1RPNjHPWuM8ryS5LDttkHdM321t85dSqaf",
        "secret": "edsk36FhrZwFVKpkdmouNmcwkAJ9XgSnE5TFHA7MqnmZ93iczDhQLK",
        "kits_to_mint": 5000*tez,
        "wtez_to_mint": 6000*tez
    },
    {
        "address": "tz1SZtpiT7KAdwZ23PDWDwdrkbe1Ci6Zzb48",
        "secret": "edsk3Vjga1rxDHC9Bns7EN4CWkpaYoHwnHBPJMdERPtytxcx7A78PG",
        "kits_to_mint": 10000*tez,
        "wtez_to_mint": 10001*tez
    },
    {
        "address": "tz1UgD3EScpgxVNVFQ3BKWspGMBzjCAUBK8z",
        "secret": "edsk3tP2tJieBNsRTqaq8edcfekPxALp3W5F2Ef4tLecqf6XG66m5F",
        "kits_to_mint": 2000*tez,
        "wtez_to_mint": 5000*tez
    }
]

The following block reveals accounts, which is necessary if they are new (as is the case on Flextesa).

In [14]:
for test_account in test_accounts:
    # Send 11000 ꜩ to each test account
    alice_ptz.transaction(destination=test_account["address"], amount=int(1.1e10)).autofill().sign().send(min_confirmations=1)
    new_ptz = tz.pytezos.using(TEZOS_RPC, test_account["secret"])
    new_ptz.reveal().autofill().sign().inject(min_confirmations=1)
    test_account["ptz"] = new_ptz  # Store the PyTezos client for later use

Operation ooosiJELeAVFfgcsdyaZjHtXc6MkoP3wFCwwu7D7Hei1qKxd13n is still in mempool
Current level: 47 (max 167)
Sleep 5 seconds until block BLw6MBXGMjNiteJR9a5RzbDtSW2xhDTyix4HBP5XzuaokM1yS4E is superseded
Found new block BLAn49ZV6teUmjm17zHQcDy9ncbrB2qiJoVqEzwZ9SNvU2wm37E (0 sec delay)
Operation ooosiJELeAVFfgcsdyaZjHtXc6MkoP3wFCwwu7D7Hei1qKxd13n has left mempool
Operation ooosiJELeAVFfgcsdyaZjHtXc6MkoP3wFCwwu7D7Hei1qKxd13n has been included to block BLAn49ZV6teUmjm17zHQcDy9ncbrB2qiJoVqEzwZ9SNvU2wm37E
Operation ooosiJELeAVFfgcsdyaZjHtXc6MkoP3wFCwwu7D7Hei1qKxd13n has 1/1 confirmations
Operation ooTgxHh6fgG7wWGWpiqxyBCMnf1TcL4JqATNncekJ39pF6ahG9S is still in mempool
Current level: 48 (max 53)
Sleep 4 seconds until block BLAn49ZV6teUmjm17zHQcDy9ncbrB2qiJoVqEzwZ9SNvU2wm37E is superseded
Found new block BLD3sG5PzTb19Dpq4Q44aCBZeWPfUSKE1aaBKLBSDShp5B4yVcd (0 sec delay)
Operation ooTgxHh6fgG7wWGWpiqxyBCMnf1TcL4JqATNncekJ39pF6ahG9S has left mempool
Operation ooTgxHh6fgG7wWGWpiqxyBCMnf1TcL4JqATN

In [15]:
for account in test_accounts:
    do_it(account["ptz"], wtez.deposit(), amount=int(account["wtez_to_mint"]))
    add_operator_fa2(account["ptz"], ch.address, wtez, 2)    

Operation op6nLyd5psXxXV7bRKYxeBKqdYGnq9bgx3heYCZjzmjzoQoyrbB is still in mempool
Current level: 57 (max 177)
Sleep 4 seconds until block BMJ1d1zmS9qnqXV2KekzMNdMFDLGHt1qdjysY358TZzaf642DzZ is superseded
Found new block BKuZ6171n57v5cJHMsQiNam4Cq9mAGyrs2zip7wpFqhs1EUNhLB (0 sec delay)
Operation op6nLyd5psXxXV7bRKYxeBKqdYGnq9bgx3heYCZjzmjzoQoyrbB has left mempool
Operation op6nLyd5psXxXV7bRKYxeBKqdYGnq9bgx3heYCZjzmjzoQoyrbB has been included to block BKuZ6171n57v5cJHMsQiNam4Cq9mAGyrs2zip7wpFqhs1EUNhLB
Operation op6nLyd5psXxXV7bRKYxeBKqdYGnq9bgx3heYCZjzmjzoQoyrbB has 1/1 confirmations
Operation opMXwuJos1bQRnxfAD2bHJ6us4ymLKxGLBDLLsVyxZgNz1Y7c7R is still in mempool
Current level: 58 (max 178)
Sleep 4 seconds until block BKuZ6171n57v5cJHMsQiNam4Cq9mAGyrs2zip7wpFqhs1EUNhLB is superseded
Found new block BLWF9Ba7iebiLABSF2JzsekD8vXTnxaimYjf9qWPPGkTkwv2AzJ (3 sec delay)
Operation opMXwuJos1bQRnxfAD2bHJ6us4ymLKxGLBDLLsVyxZgNz1Y7c7R has left mempool
Operation opMXwuJos1bQRnxfAD2bHJ6us4ymLKxGLBD

This is the big map used as a ledger for Wtez tokens:

In [16]:
wtez_ledger_id = wtez.storage["fa2_state"]["ledger"]()
alice_ptz.shell.head.context.big_maps[wtez_ledger_id]()

[{'int': '1500000000'},
 {'int': '6000000000'},
 {'int': '1200000000'},
 {'int': '10001000000'},
 {'int': '5000000000'}]

In [17]:
for account in test_accounts:
    do_it(account["ptz"], ch.create_burrow((0, None, int(1e6))))

Operation ooJVbag73w3vb2du4hCoqCaVJiJuQmdfbcwVLcriUxhpVmpHYX9 is still in mempool
Current level: 67 (max 187)
Sleep 4 seconds until block BMVojB8jX5brx5Xk8kJMgTYwUKwguN5UJPjW3hKyM8SECm4qhYe is superseded
Found new block BLJpboPGx9xNqei7RayoDciqPPauMyaTHAiaCQf6XsyCcwnUNc3 (0 sec delay)
Operation ooJVbag73w3vb2du4hCoqCaVJiJuQmdfbcwVLcriUxhpVmpHYX9 has left mempool
Operation ooJVbag73w3vb2du4hCoqCaVJiJuQmdfbcwVLcriUxhpVmpHYX9 has been included to block BLJpboPGx9xNqei7RayoDciqPPauMyaTHAiaCQf6XsyCcwnUNc3
Operation ooJVbag73w3vb2du4hCoqCaVJiJuQmdfbcwVLcriUxhpVmpHYX9 has 1/1 confirmations
Operation opKJkJwvKbmvVJRt6WzzkhsGDyDvUhV7egEtqXvMJdaGASjWspu is still in mempool
Current level: 68 (max 188)
Sleep 4 seconds until block BLJpboPGx9xNqei7RayoDciqPPauMyaTHAiaCQf6XsyCcwnUNc3 is superseded
Found new block BMFLjrbgiByxxx2AuAUMpcdLHS4uTYdEg4bnD2VPPBY1u9A1Y2Y (0 sec delay)
Operation opKJkJwvKbmvVJRt6WzzkhsGDyDvUhV7egEtqXvMJdaGASjWspu has left mempool
Operation opKJkJwvKbmvVJRt6WzzkhsGDyDvUhV7egE

All burrows need to have enough collateral (210% by default) to mint kits. Before playing with the oracle price or Checker's CFMM, we're going to mint various amount of kits and use the smart contract's views to get information about each burrow's state.

In [18]:
for account in test_accounts:
    do_it(account["ptz"], ch.deposit_collateral(0, int(account["wtez_to_mint"]-2*tez)))
    do_it(account["ptz"], ch.mint_kit(0, int((10/21)*(account["kits_to_mint"]-2*tez))))

Operation oohYXBrK5vcaVTDV5GDChHb828YhkdSJNDu43U3vC4V6qCZBqzV is still in mempool
Current level: 72 (max 192)
Sleep 4 seconds until block BLpNKi9h2eaKnhsEB5Zw6pFXJ7w8qSchDQafn1BUDF7Xm5qcsx1 is superseded
Found new block BLTmckDr4qvjaDs8owiB7RKfuH5WKWJ3jePZ1xd8Lv1ahbfsz4N (0 sec delay)
Operation oohYXBrK5vcaVTDV5GDChHb828YhkdSJNDu43U3vC4V6qCZBqzV has left mempool
Operation oohYXBrK5vcaVTDV5GDChHb828YhkdSJNDu43U3vC4V6qCZBqzV has been included to block BLTmckDr4qvjaDs8owiB7RKfuH5WKWJ3jePZ1xd8Lv1ahbfsz4N
Operation oohYXBrK5vcaVTDV5GDChHb828YhkdSJNDu43U3vC4V6qCZBqzV has 1/1 confirmations
Operation opFUx47HzXykidCLb6FuSmKd747S5PXoGU95KeA2uzP4AEf8XrQ is still in mempool
Current level: 73 (max 193)
Sleep 4 seconds until block BLTmckDr4qvjaDs8owiB7RKfuH5WKWJ3jePZ1xd8Lv1ahbfsz4N is superseded
Found new block BMTz7ytWXYKedbQbqo2VsC4YwifgDpyXQxpE4jntnjmoXthHeMf (0 sec delay)
Operation opFUx47HzXykidCLb6FuSmKd747S5PXoGU95KeA2uzP4AEf8XrQ has left mempool
Operation opFUx47HzXykidCLb6FuSmKd747S5PXoGU9

Checker's ledger gives us some information about the kits minted by every actor. Later on, it will also list the CFMM's liquidity tokens (which have a different FA2 token ID), so keep in mind that we only use it as a hint.

In [19]:
kits_ledger_id = ch.storage["deployment_state"]["sealed"]["fa2_state"]["ledger"]()
alice_ptz.shell.head.context.big_maps[kits_ledger_id]()

[{'int': '951428571'},
 {'int': '475238095'},
 {'int': '475238095'},
 {'int': '2380000000'},
 {'int': '4760952380'}]

How close is everyone from its collateral limit? We can know this by using the `burrow_max_mintable_kit` view and comparing to the quantity we know we minted.

In [20]:
kits_ledger = ch.storage["deployment_state"]["sealed"]["fa2_state"]["ledger"]

In [21]:
for account in test_accounts:
    # This is actually the amount than an address holds, but we didn't transfer any kit so far.
    minted = kits_ledger[(0, account["address"])]() / tez
    max_mintable = ch.burrow_max_mintable_kit(account["address"], 0).onchain_view() / tez
    is_overburrowed = ch.is_burrow_overburrowed(account["address"], 0).onchain_view()
    print(minted, "\t\t", max_mintable, "\t\t", is_overburrowed)

475.238095 		 570.47619 		 False
475.238095 		 713.333333 		 False
2380.0 		 2856.190476 		 False
4760.95238 		 4761.428571 		 False
951.428571 		 2380.0 		 False


Checker's state is updated through the `touch` entrypoint. A call to `touch` can be a bit costly, so the system rewards whoever calls it. To keep things simple here, we're going to have `alice` call this entrypoint every time.

In the ideal situation, `touch` is called in each block. However, all internal computations are time-dependent and the actual frequency of these calls does not change the system's behaviour.

In [22]:
_ = do_it(alice_ptz, ch.touch())

Operation opQhrvqJDY46F3MoUhbe7wpzgHWqtSJk5M8s4ecgFBZXjuy94GR is still in mempool
Current level: 82 (max 202)
Sleep 3 seconds until block BL1N1oxBXK91oZusrbXoZWhh5Bu7VvRLtiiwx3KW8ELLeSaKkc2 is superseded
Found new block BKoyMZ5uokJK8dNC5yhozkVHkqjbtSF3PYBkJzGBx48pTS6XXs5 (0 sec delay)
Operation opQhrvqJDY46F3MoUhbe7wpzgHWqtSJk5M8s4ecgFBZXjuy94GR has left mempool
Operation opQhrvqJDY46F3MoUhbe7wpzgHWqtSJk5M8s4ecgFBZXjuy94GR has been included to block BKoyMZ5uokJK8dNC5yhozkVHkqjbtSF3PYBkJzGBx48pTS6XXs5
Operation opQhrvqJDY46F3MoUhbe7wpzgHWqtSJk5M8s4ecgFBZXjuy94GR has 1/1 confirmations


Checker adapts the quantity of mintable kits slowly over time, depending on:
* the total quantity of minted kits;
* the index given by the oracle;
* the price of kit in tez in the previous, given by Checker's CFMM.

In [23]:
ch.parameters()

{'burrow_fee_index': 18446744517970629048,
 'circulating_kit': 9043110691,
 'drift': 0,
 'drift_derivative': 0,
 'imbalance_index': 18446744075576375954,
 'index': 18446744073709551616,
 'last_touched': 1687452023,
 'outstanding_kit': 9042857358,
 'protected_index': 18446744073709551616,
 'q': 18446744073709551616,
 'target': 18446744073709551616}

In [26]:
ch.cfmm()

{'ctok': 1,
 'kit': 218,
 'kit_in_ctok_in_prev_block': {'den': 1000000, 'num': 1000000},
 'last_level': 83,
 'lqt': 1}

So far, we added no liquidity to the CFMM. However, in its default configuration (decided at compile-time), Checker mints kits to reward `touch` caller and the CFMM liquidity providers. Let's call `touch` a few more times.

In [27]:
_ = do_it(alice_ptz, ch.touch())
_ = do_it(alice_ptz, ch.touch())
_ = do_it(alice_ptz, ch.touch())

Operation opXTCBE8ztQ4hvo9SapXXQTwntJGm9mEAgRuE491CLnBocxvEqd is still in mempool
Current level: 97 (max 217)
Sleep 4 seconds until block BLjq5RX53kKvWr3aj9stD7t2RtgVJDMLZnEeAh4Bh8ukteyYtMh is superseded
Found new block BMR85tt58JPYT3PLbG6tW81dWaaAkkVUmuw5TFFMrGZTi6Reg8b (0 sec delay)
Operation opXTCBE8ztQ4hvo9SapXXQTwntJGm9mEAgRuE491CLnBocxvEqd has left mempool
Operation opXTCBE8ztQ4hvo9SapXXQTwntJGm9mEAgRuE491CLnBocxvEqd has been included to block BMR85tt58JPYT3PLbG6tW81dWaaAkkVUmuw5TFFMrGZTi6Reg8b
Operation opXTCBE8ztQ4hvo9SapXXQTwntJGm9mEAgRuE491CLnBocxvEqd has 1/1 confirmations
Operation opE9U49VhqVsMvv95RCBR81B8HgFeJd7t9jmRR9G5kLZ4gTubmr is still in mempool
Current level: 98 (max 218)
Sleep 4 seconds until block BMR85tt58JPYT3PLbG6tW81dWaaAkkVUmuw5TFFMrGZTi6Reg8b is superseded
Found new block BLxwTsdy5kiREPNbcURWDmHt32KpYV7ZyuXF43dm2aCcgpkxBe8 (0 sec delay)
Operation opE9U49VhqVsMvv95RCBR81B8HgFeJd7t9jmRR9G5kLZ4gTubmr has left mempool
Operation opE9U49VhqVsMvv95RCBR81B8HgFeJd7t9j

As we never added more liquidity than the initial `1e-6ctez`, this upsets the system's balance.

In [44]:
ch.cfmm()

{'ctok': 1,
 'kit': 345,
 'kit_in_ctok_in_prev_block': {'den': 334000000, 'num': 1000000},
 'last_level': 106,
 'lqt': 1}

The new target computation is a roughly given by:

In [29]:
previous_cfmm_state = ch.cfmm()["kit_in_ctok_in_prev_block"]
ch.index() * ch.q() / (previous_cfmm_state["num"]/previous_cfmm_state["den"] * 2**64)

5.681597174703556e+21

As the target is very large, Checker starts compensating by slowly raising the `drift`, which will slowly penalize minters and force them to fix the price:

In [30]:
ch.drift()

2471110

In [31]:
_ = do_it(alice_ptz, ch.touch())

Operation ooLrqazS2xF1H2ZFqu3YLnGDp4a2DiddBKRVuekVtCuKX3L4nKy is still in mempool
Current level: 100 (max 220)
Sleep 3 seconds until block BLvEifKYVEupZgjx3ePcuzmZVsavqp8TqrKpiUEgSS5TDUQJQaN is superseded
Found new block BLpM84WezweV89agRob8F147fdxnEJ3MMXDemwUTrZ47JGL7Z1T (0 sec delay)
Operation ooLrqazS2xF1H2ZFqu3YLnGDp4a2DiddBKRVuekVtCuKX3L4nKy has left mempool
Operation ooLrqazS2xF1H2ZFqu3YLnGDp4a2DiddBKRVuekVtCuKX3L4nKy has been included to block BLpM84WezweV89agRob8F147fdxnEJ3MMXDemwUTrZ47JGL7Z1T
Operation ooLrqazS2xF1H2ZFqu3YLnGDp4a2DiddBKRVuekVtCuKX3L4nKy has 1/1 confirmations


The drift will keep on increasing 

In [32]:
ch.drift()

7413330

How does that affect the minters? Despite minting a few fresh kits for `alice` and the CFMM, Checker does not immediately overcompensate, and the burrows are still not overburrowed:

In [33]:
for account in test_accounts:
    # This is actually the amount than an address holds, but we didn't transfer any kit so far.
    minted = kits_ledger[(0, account["address"])]() / tez
    max_mintable = ch.burrow_max_mintable_kit(account["address"], 0).onchain_view() / tez
    is_overburrowed = ch.is_burrow_overburrowed(account["address"], 0).onchain_view()
    print(minted, "\t\t", max_mintable, "\t\t", is_overburrowed)

475.238095 		 570.47619 		 False
475.238095 		 713.333333 		 False
2380.0 		 2856.190476 		 False
4760.95238 		 4761.428571 		 False
951.428571 		 2379.999999 		 False


If you compare with the previous call, all the burrows can mint slightly less new kits than before. However, none of them are overburrowed for now. With the current parameters, we'd have to wait a few hours for the 4th one to become overburrowed. Moreover, even after this one becomes overburrowed, it won't immediately become liquidatable.

# TODO: time estimations, plots

# TODO: oracle

Let's refresh Checker's state:

In [34]:
_ = do_it(alice_ptz, ch.touch())
ch.storage()

Operation opGhaBXYqEWJb7rnv6ZRRNZP3jZUhFDPRkBnaehtfPsNxC4Nioz is still in mempool
Current level: 101 (max 221)
Sleep 3 seconds until block BLpM84WezweV89agRob8F147fdxnEJ3MMXDemwUTrZ47JGL7Z1T is superseded
Found new block BLhYbo84YxMnYPTjq2LP9nRphbZyHXe4SrfbdpWiy6j6psrvFPG (0 sec delay)
Operation opGhaBXYqEWJb7rnv6ZRRNZP3jZUhFDPRkBnaehtfPsNxC4Nioz has left mempool
Operation opGhaBXYqEWJb7rnv6ZRRNZP3jZUhFDPRkBnaehtfPsNxC4Nioz has been included to block BLhYbo84YxMnYPTjq2LP9nRphbZyHXe4SrfbdpWiy6j6psrvFPG
Operation opGhaBXYqEWJb7rnv6ZRRNZP3jZUhFDPRkBnaehtfPsNxC4Nioz has 1/1 confirmations


{'lazy_functions': 16,
 'metadata': 17,
 'deployment_state': {'sealed': {'burrows': 18,
   'cfmm': {'ctok': 1,
    'kit': 323,
    'kit_in_ctok_in_prev_block': {'den': 318000000, 'num': 1000000},
    'last_level': 102,
    'lqt': 1},
   'external_contracts': {'collateral_fa2': 'KT1FJDtuFLRjZajFWVwZZhfd2xxADmqc3qoc',
    'ctez_cfmm': 'KT1J5WbrrzxCAMx3YEDGNEEmu7YwuPvxTynj',
    'ctok_fa2': 'KT1JzZbLsSidp71Fq8VNHr4eq3BiDqS5U5sT',
    'oracle': 'KT1C1SsTheFJJD96uDFNhCqBVyrEgZvikosQ'},
   'fa2_state': {'ledger': 19, 'operators': 20},
   'last_ctez_in_tez': {'den': 5043456793138493339171717132818382567050206626619577173497381555743452386751642958261026080625269202023248382759272448,
    'num': 5043456793138493339171717132818382567050206626619577173497381555743452386751642958261026080625269202023248382759272448},
   'last_index': 18446744073709551616,
   'liquidation_auctions': {'avl_storage': {'last_ptr': 7, 'mem': 21},
    'burrow_slices': 22,
    'completed_auctions': None,
    'current_au

What happens if the price of the oracle changes suddendly? Let's simulate a 10% price reduction of the collateral:

In [35]:
_ = do_it(alice_ptz, oracle.default((10,9)))

Operation ooGASM4eTHz135pTtmcUjccitVggXDv1X1VadpV3kpgQzH2i8BD is still in mempool
Current level: 102 (max 222)
Sleep 4 seconds until block BLhYbo84YxMnYPTjq2LP9nRphbZyHXe4SrfbdpWiy6j6psrvFPG is superseded
Found new block BLbURjNCBUF7QxPLkTRFzneou2uN5wWYhGw8bexzvtoC1Jrbe8u (0 sec delay)
Operation ooGASM4eTHz135pTtmcUjccitVggXDv1X1VadpV3kpgQzH2i8BD has left mempool
Operation ooGASM4eTHz135pTtmcUjccitVggXDv1X1VadpV3kpgQzH2i8BD has been included to block BLbURjNCBUF7QxPLkTRFzneou2uN5wWYhGw8bexzvtoC1Jrbe8u
Operation ooGASM4eTHz135pTtmcUjccitVggXDv1X1VadpV3kpgQzH2i8BD has 1/1 confirmations


For Checker to adapt, once again, we have to call the `touch` entrypoint.

In [36]:
_ = do_it(alice_ptz, ch.touch())
ch.parameters()

Operation ooW8N4dsWZ6jAAXmh9gMHsF48VGamPN6YvrmkmtUArxF84wTm1A is still in mempool
Current level: 103 (max 223)
Sleep 3 seconds until block BLbURjNCBUF7QxPLkTRFzneou2uN5wWYhGw8bexzvtoC1Jrbe8u is superseded
Found new block BKj7Ej4aDV911oALnXyQ4MfsfTv3fgEh6T4cFfffuaSJTMeFY2j (0 sec delay)
Operation ooW8N4dsWZ6jAAXmh9gMHsF48VGamPN6YvrmkmtUArxF84wTm1A has left mempool
Operation ooW8N4dsWZ6jAAXmh9gMHsF48VGamPN6YvrmkmtUArxF84wTm1A has been included to block BKj7Ej4aDV911oALnXyQ4MfsfTv3fgEh6T4cFfffuaSJTMeFY2j
Operation ooW8N4dsWZ6jAAXmh9gMHsF48VGamPN6YvrmkmtUArxF84wTm1A has 1/1 confirmations


{'burrow_fee_index': 18446744763483336415,
 'circulating_kit': 9043250804,
 'drift': 22239990,
 'drift_derivative': 1235555,
 'imbalance_index': 18446744077043701208,
 'index': 20496382304121724017,
 'last_touched': 1687452107,
 'outstanding_kit': 9042857474,
 'protected_index': 18447973856647798919,
 'q': 18446744073910535229,
 'target': 6517849572781722447332}

Note that new `index` value is very different from the previous one, but not the `protected_index`. Also compare the values for `max_mintable_kit` from the previous ones:

In [37]:
account1 = test_accounts[0]["address"]
ch.burrow_max_mintable_kit(account1, 0).onchain_view() / tez

513.428571

In [38]:
account2 = test_accounts[3]["address"]
ch.burrow_max_mintable_kit(account2, 0).onchain_view() / tez

4285.285714

In [39]:
ch.is_burrow_overburrowed(account2, 0).onchain_view()

True

In [40]:
ch.is_burrow_liquidatable(account2, 0).onchain_view()

False

Finally, let's reduce the price some more and see what happens to the various `index` parameters:

In [41]:
_ = do_it(alice_ptz, oracle.default((10,7)))
_ = do_it(alice_ptz, ch.touch())

Operation opNw43oANE2dtoKN9boPUqEX7SqR1ECwLfudMKoVTiDSeJLMyJe is still in mempool
Current level: 104 (max 224)
Sleep 4 seconds until block BKj7Ej4aDV911oALnXyQ4MfsfTv3fgEh6T4cFfffuaSJTMeFY2j is superseded
Found new block BLCs5SwxieF7BrFVUNsfNthwmgy38NJVpWTANyBkM5H1wss79Wi (0 sec delay)
Operation opNw43oANE2dtoKN9boPUqEX7SqR1ECwLfudMKoVTiDSeJLMyJe has left mempool
Operation opNw43oANE2dtoKN9boPUqEX7SqR1ECwLfudMKoVTiDSeJLMyJe has been included to block BLCs5SwxieF7BrFVUNsfNthwmgy38NJVpWTANyBkM5H1wss79Wi
Operation opNw43oANE2dtoKN9boPUqEX7SqR1ECwLfudMKoVTiDSeJLMyJe has 1/1 confirmations
Operation opD2R6QcECyiS3Hm16ALHL6bCLa59pdVMAf9zNCmwWRDhsWNjLh is still in mempool
Current level: 105 (max 225)
Sleep 3 seconds until block BLCs5SwxieF7BrFVUNsfNthwmgy38NJVpWTANyBkM5H1wss79Wi is superseded
Found new block BMVfDpefF9PKS2zhBKqYsnmPrx63EQUVNXA6vig1ZdKyZ5eQdjY (0 sec delay)
Operation opD2R6QcECyiS3Hm16ALHL6bCLa59pdVMAf9zNCmwWRDhsWNjLh has left mempool
Operation opD2R6QcECyiS3Hm16ALHL6bCLa59pdVM

In [42]:
ch.parameters()

{'burrow_fee_index': 18446744786865499259,
 'circulating_kit': 9043264148,
 'drift': 32124430,
 'drift_derivative': 1235555,
 'imbalance_index': 18446744077201420699,
 'index': 26352491533870788022,
 'last_touched': 1687452115,
 'outstanding_kit': 9042857485,
 'protected_index': 18449203721571575438,
 'q': 18446744074127992909,
 'target': 8511854765633345299161}

In [43]:
ch.is_burrow_liquidatable(account2, 0).onchain_view()

False

As we can see, even if this burrow is now really lacking collateral, it is not yet liquidatable. Checker protects the minters from brutal oracle (and price) manipulation by using a `protected_index` which is not immediately catching up to the `index`. In `src/parameters.mligo`, you can see that the limit for minting kits is computed by taking the _maximum_ of the `index` and the `protected_index`, while the condition for liquidation is computed with the _minimum_.

While this choice protects the minters, it can be detrimental to liquidators in case of a sudden crash. Make sure to compare the influence of various parameters before using this in production.

TODO: plots and time estimations as well.